In [11]:
from kafka import KafkaProducer,KafkaClient
from kafka.errors import KafkaError
import time
import json
import websocket
import logging
import re
#LOCAL IMPORTS
import config as myCred
from confluent_kafka.admin import AdminClient
import hashlib

In [17]:
def onOpen(ws):
    print("connection Opened")
    auth_data={
        "action": "authenticate",
        "data": {"key_id": myCred.API_KEY, "secret_key": myCred.SECRET_KEY}
    }
    
    ws.send(json.dumps(auth_data))
    
    listenMsg = {
        "action": "listen", 
        "data": {"streams": ["T.TSLA","T.AAPL","T.MSFT"]}
    }

    ws.send(json.dumps(listenMsg))
    
def onMsg(ws,message):
    if re.match('^.*authorized.*$',message):
        print(message)
        return
    else:
        if(time.time()-startTime)<10:
            print(message)
            try:
                producer.send(topicName, value=message.encode('utf-8'))
                return True
            except Exception as ex:
                logging.error(str(ex))
        else:
            producer.close()
            ws.close()
            
def onClose(ws):
    print("connection Closed")

In [19]:
if __name__ == "__main__":
    #creating producer log file
    logging.basicConfig(filename="producer_log.log", level=logging.INFO)
    
    #unit test to check if zookeeper an kafka are running properly
    conf = {'bootstrap.servers':'localhost:9092'}
    admin_client = AdminClient(conf)
    topics = admin_client.list_topics().topics
    if not topics: 
        logging.error(RuntimeError())
    else:
        logging.info(topics)
    #websocket for data stream
    startTime =time.time()
    socket = "wss://data.alpaca.markets/stream"
    ws = websocket.WebSocketApp(socket,on_open=onOpen, on_close=onClose, on_message=onMsg)
    
    try:
        producer=KafkaProducer(bootstrap_servers=" localhost:9092")
        topicName="test-topic"
    except KafkaError:
        logging.error(KafkaError+'\n')
    ws.run_forever()

connection Opened
{"stream":"authorization","data":{"action":"authenticate","status":"authorized"}}
{"stream":"listening","data":{"streams":["T.TSLA","T.AAPL","T.MSFT"]}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"727","x":3,"p":462.3,"s":3,"t":1597336175123000000,"c":[14,37,41],"z":3}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"2950","x":2,"p":462.24,"s":2,"t":1597336178144000000,"c":[37],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"887","x":17,"p":210.98,"s":100,"t":1597336179515000000,"c":[14,41],"z":3}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"2951","x":2,"p":462.28,"s":100,"t":1597336180614000000,"c":[14,41],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"313","x":3,"p":211,"s":11,"t":1597336180615000000,"c":[14,37,41],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"314","x":3,"p":211,"s":42,"t":1597336180616000000,"c":[14,37,41],"z":3}}
